In [1]:
import geojson
import pandas as pd
import shapely.wkt
from shapely import geometry
import pickle as pkl
import geopandas as gpd
from random import sample
from tqdm.notebook import tqdm
import csv
import math
import numpy as np
import pyproj
from shapely.ops import transform



In [2]:
datasetName='slender_panda'
country='bangladesh'
stateSeparatedFile='googletest_state_sampled_'+country+datasetName
datasetFile='googletest_state_sampled_'+datasetName
maxKilnStateSample=500
maxNonKilnStateSample=250
minNonKilnStateSample=50
randomSample=500
oversampleStates=['F.A.T.A','N.W.F.P','Punjab']
oversampleModifier=1.5

In [3]:
#Load Datasets
highDataset={'idx':{},'geometry':{},'prediction':{},'area':{}}
restDataset={'idx':{},'geometry':{},'prediction':{},'area':{}}
totalDataset=pd.read_csv('../Predicted_Datasets/'+datasetName+'_results.csv').to_dict()
highcount=0
lowcount=0
high_pred=0
for idx in tqdm(totalDataset['prediction']):
    if totalDataset['prediction'][idx]>high_pred:
        high_pred=totalDataset['prediction'][idx]
    if totalDataset['prediction'][idx] >= .90:
        highDataset['idx'][highcount]=totalDataset['idx'][idx]
        highDataset['geometry'][highcount]=totalDataset['geometry'][idx]
        highDataset['prediction'][highcount]=totalDataset['prediction'][idx]
        highcount+=1
    else:
        restDataset['idx'][lowcount]=totalDataset['idx'][idx]
        restDataset['geometry'][lowcount]=totalDataset['geometry'][idx]
        restDataset['prediction'][lowcount]=totalDataset['prediction'][idx]
        lowcount+=1

  0%|          | 0/445394 [00:00<?, ?it/s]

In [4]:
#Create shapely brick belt
with open('../GeoJSONS/Brick_Belt.geojson') as f:
    brick_belt = geojson.load(f) 
with open('../GeoJSONS/Bangladesh_States.geojson') as f:
    states=geojson.load(f)
shapelyBelt=geometry.shape(brick_belt['features'][0]['geometry'])

In [5]:
#Prepare to make a geodataframe from the geometry of the country states, also take the time to calculate non-belt max area
shapelyStates={}
togpd=[]
names=[]
within={}
max_area=0
for x in states['features']:
    if shapelyBelt.intersects(geometry.shape(x['geometry'])):
        within = shapelyBelt.intersection(geometry.shape(x['geometry'])).area/geometry.shape(x['geometry']).area >= .25
    else:
        within=False
    shapelyStates[x['properties']['NAME_1']]={
        'geometry':geometry.shape(x['geometry']),
        'within':within,
        'tiles':[]
    }
    if not shapelyStates[x['properties']['NAME_1']]['within']:
        if max_area<shapelyStates[x['properties']['NAME_1']]['geometry'].area:
            max_area=shapelyStates[x['properties']['NAME_1']]['geometry'].area
    togpd.append(geometry.shape(x['geometry']))
    names.append(x['properties']['NAME_1'])

In [6]:
#Create state geodataframe
gdf = gpd.GeoDataFrame(data={'name':names}, crs='epsg:4326', geometry=togpd)

In [7]:

#"""
#Iterate through all of our keys and obtain the geometric center and the respective tile_keys
centers=[]
tile_idxs=[]
geoms=[]

source = pyproj.CRS('EPSG:3857')
to = pyproj.CRS('EPSG:4326')
project = pyproj.Transformer.from_crs(source, to, always_xy=True).transform

for x in tqdm(highDataset['geometry'].keys()):
    shapegeom=geometry.shape(eval(highDataset['geometry'][x])['geometry'])
    transformedgeom = transform(project, shapegeom)
    centers.append(transformedgeom.centroid)
    tile_idxs.append(highDataset['idx'][x])
    geoms.append(shapegeom)
    
#Create the tile geodataframe from the center points 
gdfp = gpd.GeoDataFrame(data={'img_idx':tile_idxs,'idx':range(len(tile_idxs)),'geom':geoms},crs='epsg:4326', geometry=centers)
#"""

  0%|          | 0/1877 [00:00<?, ?it/s]

In [8]:
#"""
#Create and save the joined point-state geometry as a csv
ans = gpd.tools.sjoin(gdfp, gdf, predicate="within", how='left')
ans.drop('geometry',axis=1).drop('index_right',axis=1).to_csv('State_Separated_Datasets/'+stateSeparatedFile+'.csv')
#"""

In [9]:
#load ans
ans=pd.read_csv('State_Separated_Datasets/'+stateSeparatedFile+'.csv').to_dict()

In [10]:
#Iterate through each tile once more and add each tile to it's respective state list
stateTiles={}
for x in shapelyStates.keys():
    stateTiles[x]=[]
for x in tqdm(range(len(ans['idx']))):
    if type(ans['name'][x])==str:
        stateTiles[ans['name'][x]].append([ans['geom'][x],ans['idx'][x]])

  0%|          | 0/1877 [00:00<?, ?it/s]

In [11]:
with open('../Predicted_Datasets/'+datasetFile, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows([['idx','geometry','prediction','area']])
    
#Iterate through each state and sample respective amounts per state, then save it to the sampled csv.    
for x in shapelyStates.keys():
    print('Working on '+x+'!')
    if x in oversampleStates:
        print('Oversampling!')
        modifier=oversampleModifier
    else:
        modifier=1
    if shapelyStates[x]['within']:
        print('Inside Brick Belt!')
        samplesize=min([len(stateTiles[x]),round(maxKilnStateSample*modifier)])
    else:
        print('Outside Brick Belt!')
        cur_area=shapelyStates[x]['geometry'].area
        samplesize=min([round(maxNonKilnStateSample*cur_area/max_area*modifier),len(stateTiles[x])])
        if round(minNonKilnStateSample*modifier) > len(stateTiles[x]):
            samplesize=len(stateTiles[x])
        else:
            samplesize=max([round(minNonKilnStateSample*modifier),samplesize])
    print('Sampling '+str(samplesize)+' tiles!')
    shapelyStates[x]['tiles']=sample(stateTiles[x],samplesize)
    rows=[]
    print('Saving!')
    for y in range(len(shapelyStates[x]['tiles'])):
        #numidx=highDataset['idx'][shapelyStates[x]['tiles'][y][1]].split('[')[1].split(']')[0]
        rows.append([highDataset['idx'][shapelyStates[x]['tiles'][y][1]],highDataset['geometry'][shapelyStates[x]['tiles'][y][1]],highDataset['prediction'][shapelyStates[x]['tiles'][y][1]],x])
    with open('../Predicted_Datasets/'+datasetFile+'_results.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(rows)
        
if randomSample:
    print('Sampling randomly!')
    sampledkeys=sample(list(restDataset['geometry'].keys()),min([len(restDataset['geometry'].keys()),randomSample]))
    print('Sampling',len(sampledkeys),'tiles!')
    rows=[]
    print('Saving!')
    for x in sampledkeys:
        #numidx=restDataset['idx'][x].split('[')[1].split(']')[0]
        rows.append([restDataset['idx'][x],restDataset['geometry'][x],restDataset['prediction'][x],'random'])
    with open('../Predicted_Datasets/'+datasetFile+'_results.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(rows)

Working on Barisal!
Inside Brick Belt!
Sampling 7 tiles!
Saving!
Working on Chittagong!
Outside Brick Belt!
Sampling 250 tiles!
Saving!
Working on Dhaka!
Outside Brick Belt!
Sampling 72 tiles!
Saving!
Working on Khulna!
Outside Brick Belt!
Sampling 168 tiles!
Saving!
Working on Mymensingh!
Outside Brick Belt!
Sampling 86 tiles!
Saving!
Working on Rajshahi!
Outside Brick Belt!
Sampling 74 tiles!
Saving!
Working on Rangpur!
Outside Brick Belt!
Sampling 27 tiles!
Saving!
Working on Sylhet!
Outside Brick Belt!
Sampling 37 tiles!
Saving!
Sampling randomly!
Sampling 500 tiles!
Saving!


In [12]:
sampledDataset=pd.read_csv('../Predicted_Datasets/'+datasetFile+'_results.csv').to_dict()

In [13]:
import ee
#ee.Authenticate()
ee.Initialize()
import folium

def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

    
folium.Map.add_ee_layer = add_ee_layer

wmap=folium.Map()
for step in tqdm(sampledDataset['geometry']):
    folium.GeoJson(ee.Geometry(eval(sampledDataset['geometry'][step])['geometry']).transform("EPSG:4326",1).getInfo(),name='test').add_to(wmap)
wmap        

  0%|          | 0/1221 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [14]:
print('hi')

hi
